In [1]:
from typing import *
import pandas as pd
from pathlib import Path
import os
import random
import sys
import re
from dotenv import load_dotenv, dotenv_values
from pydriller import Repository
import json
from multiprocessing import Pool
from itertools import chain
from pymongo import MongoClient
import collections

In [2]:
import dotenv
dotenv.load_dotenv()
ENV = dotenv.dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])

client = MongoClient("localhost", 42692)
db = client.s5_snyk_libio

In [3]:
db.mergedVuln.drop()

In [4]:
libio_vuln_data = list(db.libioVuln.find())
mvn_eco_vuln_data = list(db.mvnEcoVuln.find())
db.mergedVuln.insert_many(db.libioVuln.find())
db.mergedVuln.insert_many(db.mvnEcoVuln.find())

InsertManyResult([ObjectId('67bc44645de95ee4df78f007'), ObjectId('67bc44645de95ee4df78f008'), ObjectId('67bc44645de95ee4df78f009'), ObjectId('67bc44645de95ee4df78f00a'), ObjectId('67bc44645de95ee4df78f00b'), ObjectId('67bc44645de95ee4df78f00c'), ObjectId('67bc44645de95ee4df78f00d'), ObjectId('67bc44645de95ee4df78f00e'), ObjectId('67bc44645de95ee4df78f00f'), ObjectId('67bc44645de95ee4df78f011'), ObjectId('67bc44645de95ee4df78f012'), ObjectId('67bc44645de95ee4df78f013'), ObjectId('67bc44645de95ee4df78f014'), ObjectId('67bc44645de95ee4df78f015'), ObjectId('67bc44645de95ee4df78f016'), ObjectId('67bc44645de95ee4df78f017'), ObjectId('67bc44645de95ee4df78f018'), ObjectId('67bc44645de95ee4df78f019'), ObjectId('67bc44645de95ee4df78f01a'), ObjectId('67bc44645de95ee4df78f01b'), ObjectId('67bc44645de95ee4df78f01c'), ObjectId('67bc44645de95ee4df78f01d'), ObjectId('67bc44645de95ee4df78f01e'), ObjectId('67bc44645de95ee4df78f01f'), ObjectId('67bc44645de95ee4df78f020'), ObjectId('67bc44645de95ee4df78f0

In [5]:
len(libio_vuln_data), len(mvn_eco_vuln_data)

(6841, 829)

In [6]:
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in libio_vuln_data)))
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in mvn_eco_vuln_data)))
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in libio_vuln_data + mvn_eco_vuln_data)))

287
829
990


In [7]:
print(len(set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data)))  # 6726  some do not have CVE ID
print(len(set((x['vuln_gav'], x['cve_ref']) for x in mvn_eco_vuln_data)))  # 829
print(len(set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data + mvn_eco_vuln_data)))  # 7490

6833
829
7587


In [8]:
libio_group_by_gav_cve = collections.defaultdict(list)
for it in libio_vuln_data:
    libio_group_by_gav_cve[(it['vuln_gav'], it['cve_ref'])].append(it)

mvn_eco_group_by_gav_cve = collections.defaultdict(list)
for it in mvn_eco_vuln_data:
    mvn_eco_group_by_gav_cve[(it['vuln_gav'], it['cve_ref'])].append(it)

In [9]:
l = list(sorted(libio_group_by_gav_cve.items(), key=lambda x: len(x[1]), reverse=True))
l[0]

(('io.netty:netty-handler:4.0.21.Final', ''),
 [{'_id': ObjectId('67b73873a59659fdd4da59e5'),
   'data_src': 'snyk',
   'cve_ref': '',
   'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-IONETTY-2342128',
   'vuln_gav': 'io.netty:netty-handler:4.0.21.Final',
   'vuln_classes': ['io.netty.handler.ssl.OpenSslEngine'],
   'patches': ['https://github.com/netty/netty/commit/4581c441c99e1d5524870abca38d0ad2d553932a']},
  {'_id': ObjectId('67b73873a59659fdd4da59e7'),
   'data_src': 'snyk',
   'cve_ref': '',
   'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-IONETTY-31518',
   'vuln_gav': 'io.netty:netty-handler:4.0.21.Final',
   'vuln_classes': ['io.netty.handler.ssl.OpenSslServerContext'],
   'patches': ['https://github.com/netty/netty/commit/a1af35313cc8414109e7a411bb2401ae31046289']},
  {'_id': ObjectId('67b73873a59659fdd4da59e8'),
   'data_src': 'snyk',
   'cve_ref': '',
   'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-IONETTY-31515',
   'vuln_gav': 'io.netty:netty-handler

In [10]:
shared_gav_cve_items = list(set(libio_group_by_gav_cve.keys()).intersection(set(mvn_eco_group_by_gav_cve.keys())))
len(shared_gav_cve_items)

75

In [11]:
_rk = random.choice(shared_gav_cve_items)
libio_group_by_gav_cve[_rk], mvn_eco_group_by_gav_cve[_rk]  
# 'CVE-2021-37137', 'io.netty:netty-codec:4.1.5.Final'   mvn_eco data has the wrong cve_ref
# 'CVE-2013-7285' mvn_eco data includes 'ReflectionConverter' which is wrong
# 

([{'_id': ObjectId('67b73873a59659fdd4da69f6'),
   'data_src': 'snyk',
   'cve_ref': 'CVE-2016-5007',
   'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-ORGSPRINGFRAMEWORKSECURITY-31336',
   'vuln_gav': 'org.springframework.security:spring-security-config:4.0.4.RELEASE',
   'vuln_classes': ['org.springframework.security.config.annotation.web.configurers.CsrfConfigurer',
    'org.springframework.security.config.annotation.web.configurers.ChannelSecurityConfigurer',
    'org.springframework.security.config.http.FilterInvocationSecurityMetadataSourceParser',
    'org.springframework.security.config.annotation.web.configurers.UrlAuthorizationConfigurer',
    'org.springframework.security.config.http.FilterChainMapBeanDefinitionDecorator',
    'org.springframework.security.config.annotation.web.AbstractRequestMatcherRegistry',
    'org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter',
    'org.springframework.security.config.http.HttpConfigurat

In [12]:
import pprint

diff_class_counter = 0

for k in shared_gav_cve_items:
    x = libio_group_by_gav_cve[k]
    if len(x) > 1:
        continue
    x = x[0]
    y = mvn_eco_group_by_gav_cve[k]
    if len(y) > 1:
        continue
    y = y[0]
    
    if set(x['vuln_classes']) == set(y['vuln_classes']):
        continue

    if set(y['vuln_classes']).issubset(set(x['vuln_classes'])):
        continue

    # if set(x['vuln_classes']).issubset(set(y['vuln_classes'])):
    #     continue
        
    diff_class_counter += 1
    pprint.pprint(x)
    print("\n------------\n")
    pprint.pprint(y)
    print("\n@@@@@@@@@@@@@@@\n")

diff_class_counter

{'_id': ObjectId('67b73873a59659fdd4da5f78'),
 'cve_ref': 'CVE-2016-1000352',
 'data_src': 'snyk',
 'patches': ['https://github.com/bcgit/bc-java/commit/9385b0ebd277724b167fe1d1456e3c112112be1f'],
 'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-ORGBOUNCYCASTLE-32366',
 'vuln_classes': ['org.bouncycastle.jcajce.provider.asymmetric.ec.IESCipher',
                  'org.bouncycastle.jcajce.provider.asymmetric.ies.AlgorithmParametersSpi',
                  'org.bouncycastle.jcajce.provider.asymmetric.DH',
                  'org.bouncycastle.jcajce.provider.asymmetric.dh.IESCipher',
                  'org.bouncycastle.jce.spec.IESParameterSpec',
                  'org.bouncycastle.jcajce.provider.asymmetric.EC',
                  'org.bouncycastle.jcajce.provider.asymmetric.util.IESUtil'],
 'vuln_gav': 'org.bouncycastle:bcprov-jdk15on:1.54'}

------------

{'_id': ObjectId('67bc44645de95ee4df78f166'),
 'cve_ref': 'CVE-2016-1000352',
 'data_src': 'mvn_eco',
 'patches': ['https://github

8

have to remove modified classes that are not in the jar (libio)

In [13]:
mvn_eco_vuln_gav_cve_ref = set((x['vuln_gav'], x['cve_ref']) for x in mvn_eco_vuln_data)
len(mvn_eco_vuln_gav_cve_ref), len(list(x for x in libio_vuln_data if (x['vuln_gav'], x['cve_ref']) not in mvn_eco_vuln_gav_cve_ref))

(829, 6766)

In [14]:
libio_vuln_gav_cve_ref = set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data)
len(libio_vuln_gav_cve_ref), len(list(x for x in mvn_eco_vuln_data if (x['vuln_gav'], x['cve_ref']) not in libio_vuln_gav_cve_ref))

(6833, 754)

In [15]:
db.mergedVuln.drop()
db.mergedVuln.insert_many(libio_vuln_data)
_ = db.mergedVuln.insert_many(x for x in mvn_eco_vuln_data if (x['vuln_gav'], x['cve_ref']) not in libio_vuln_gav_cve_ref)

In [16]:
uniq_cve = set()
uniq_cve_ga = set()
uniq_cve_gav_set = set()

for it in db.mergedVuln.find():
    cve_ref = it['cve_ref']
    if cve_ref == '':
        cve_ref = it['snyk_url']
    
    assert cve_ref != '' and cve_ref is not None
    
    uniq_cve.add(cve_ref)
    uniq_cve_ga.add((cve_ref, ':'.join(it['vuln_gav'].split(':')[:-1])))
    uniq_cve_gav_set.add((cve_ref, it['vuln_gav']))

print(len(uniq_cve))  
print(len(uniq_cve_ga))
print(len(uniq_cve_gav_set))

954
992
7595
